In [1]:
%matplotlib inline

In [2]:
import os
import sys
import glob
import pandas as pd
import matplotlib.pyplot as plt
import torch
import numpy as np

In [3]:
# Add parent dir to interpreter path
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [4]:
DATA_DIRS_LIST = glob.glob("../data/*")

DATA_DIRS_LIST

['../data/german_equity', '../data/dax-euro', '../data/simulated']

In [5]:
glob.glob(os.path.join(DATA_DIRS_LIST[0], "*"))

['../data/german_equity/xFGBS',
 '../data/german_equity/xFGBM',
 '../data/german_equity/xFGBL',
 '../data/german_equity/README']

In [6]:
glob.glob(os.path.join(DATA_DIRS_LIST[1], "*"))

['../data/dax-euro/xFSXE', '../data/dax-euro/README', '../data/dax-euro/xFDAX']

In [7]:
DAX_DATA_PATH = "../data/dax-euro/xFDAX"
DAX_DATA_LIST = glob.glob(os.path.join(DAX_DATA_PATH, "*"))

In [8]:
df = pd.read_csv(DAX_DATA_LIST[30])

In [9]:
df

,Date,Time,Volume,OrderType,Side,AskPriceAfter,AskQtyAfter,AskSplitAfter,BidPriceAfter,BidQtyAfter,BidSplitAfter
0,20140516,28805.966663,2,0,-1,9679.0,17,5,9678.5,8,1
1,20140516,28805.966663,8,1,1,9679.0,17,5,9678.5,8,1
2,20140516,28805.967000,5,1,1,9679.0,17,5,9678.5,13,2
3,20140516,28805.976119,8,0,-1,9679.0,9,3,9678.5,5,1
4,20140516,28805.976119,8,-1,1,9679.0,9,3,9678.5,5,1
5,20140516,28805.976489,5,0,-1,9679.0,4,1,9677.5,1,1
6,20140516,28805.976489,5,-1,1,9679.0,4,1,9677.5,1,1
7,20140516,28806.069280,1,1,1,9679.0,4,1,9678.0,1,1
8,20140516,28806.069280,1,0,-1,9679.0,3,1,9678.0,1,1
9,20140516,28806.307145,1,1,-1,9679.0,4,2,9678.0,1,1


In [10]:
# Remove cancel orders from the dataset
df = df[df.OrderType != 0]

In [11]:
evt_times = df.Time.values

evt_types = (df.OrderType.values + 1)//2

In [12]:
day_stamp = df.Date.unique()[0]
print(day_stamp)

20140516


In [20]:
split_times_list = np.array_split(evt_times, 1000)
split_types_list = np.array_split(evt_types, 1000)
seq_lengths = [len(e) for e in split_times_list]

seq_lengths = torch.LongTensor(seq_lengths) - 1

split_times_list = [torch.from_numpy(e) for e in split_times_list]
split_types_list = [torch.from_numpy(e) for e in split_types_list]

In [17]:
from torch import nn

In [27]:
seq_times = nn.utils.rnn.pad_sequence(split_times_list, batch_first=True).to(torch.float32)
seq_times

tensor([[28805.9668, 28805.9668, 28805.9766,  ..., 30077.3672, 30077.8438,
         30077.8691],
        [30077.8691, 30077.8691, 30077.8809,  ..., 31088.0742, 31088.6445,
         31089.1465],
        [31089.9023, 31089.9023, 31090.3535,  ..., 31789.1719, 31789.1719,
         31789.1719],
        ...,
        [76558.8906, 76559.0078, 76562.6484,  ..., 77518.2344, 77518.2344,
             0.0000],
        [77518.8438, 77522.3203, 77522.3203,  ..., 78190.4688, 78190.4688,
             0.0000],
        [78190.4688, 78191.7500, 78192.7344,  ..., 79199.0000, 79199.5078,
             0.0000]])

In [28]:
seq_types = nn.utils.rnn.pad_sequence(split_types_list, batch_first=True, padding_value=2)

seq_types

tensor([[1, 1, 0,  ..., 1, 0, 1],
        [0, 1, 1,  ..., 0, 1, 1],
        [1, 1, 1,  ..., 1, 1, 0],
        ...,
        [0, 1, 1,  ..., 0, 0, 2],
        [1, 1, 1,  ..., 1, 0, 2],
        [0, 1, 0,  ..., 1, 1, 2]])

In [29]:
from models import HawkesDecayRNN
from train_functions import train_decayrnn

In [34]:
process_dim = 2
hidden_size = 64

In [35]:
EPOCHS = 3
BATCH_SIZE = 32

In [36]:
model = HawkesDecayRNN(process_dim, hidden_size)

In [37]:
from torch import optim

In [38]:
optimizer = optim.Adam(model.parameters(), lr=0.005)

In [39]:
loss = train_decayrnn(model, optimizer, seq_times, seq_types, seq_lengths, 0.,
                      BATCH_SIZE, EPOCHS, use_jupyter=True)

Batch size 32
Number of epochs 3
Train size: 250


KeyboardInterrupt: 